# Instructions
If you would like to skip processing the whole dataset as it can take a long time, you can use the preprocessed results data and models available in Results and Models directories.

To run this reduced analysis, run the following sections of this notebook:
1. Imports
1. Definitions for processing functions
1. Save & load (only the 4th cell)
1. Analyze results

### Imports

In [1]:
%load_ext autoreload
%autoreload 2

import pathlib
import pickle
import random
import time

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import torch
import torch_geometric.utils as tg_utils
from torch_geometric.loader import DataLoader
from tqdm.notebook import tqdm, trange

from my_graphs_dataset import GraphDataset
from MIDS_dataset import MIDSDataset, MIDSLabelsDataset, MIDSProbabilitiesDataset
from MIDS_script import generate_model


In [4]:
root = pathlib.Path().cwd()  # For Jupyter notebook.

### Definitions for testing functions

In [3]:
def load_dataset(root):
    # Set up parameters.
    seed = 42
    selected_graph_sizes = {
        # "26-50_mix_100": -1,
        "03-25_mix_750": 10
    }
    split = (0.6, 0.2)
    batch_size = 1

    # Get the dataset.
    loader = GraphDataset(selection=selected_graph_sizes, seed=seed)
    prob_dataset = MIDSProbabilitiesDataset(root / "Dataset", loader)
    labels_dataset = MIDSLabelsDataset(root / "Dataset", loader, selected_extra_feature="predicted_probability")

    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    prob_dataset, perm = prob_dataset.shuffle(return_perm=True)
    labels_dataset = labels_dataset.index_select(perm) # type: ignore
    assert isinstance(prob_dataset, MIDSDataset)
    assert isinstance(labels_dataset, MIDSDataset)

    # Flexible dataset splitting. Can be split to train/test or train/val/test.
    if isinstance(split, tuple):
        train_size, val_size = split
        train_size = round(train_size * len(prob_dataset))
        val_size = round(val_size * len(prob_dataset))
    else:
        train_size = round(split * len(prob_dataset))
        val_size = len(prob_dataset) - train_size

    test_probs = prob_dataset[train_size + val_size:]
    test_labels = labels_dataset[train_size + val_size:]

    # Batch and load data.
    prob_loader = DataLoader(test_probs, batch_size, shuffle=False, pin_memory=True)  # type: ignore
    labels_loader = DataLoader(test_labels, batch_size, shuffle=False, pin_memory=True)  # type: ignore
    prob_data = [test_batch for test_batch in prob_loader]
    labels_data = [test_batch for test_batch in labels_loader]

    return prob_data, labels_data, labels_dataset.num_features

In [ ]:
from Utilities.mids_utils import check_MIDS

def run_GNN(root, prob_data, label_data, num_features):
    device = "cpu"  # "cuda" if torch.cuda.is_available() else "cpu"

    # Load the probability model.
    prob_model = torch.load(root / "Models" / "prob_model_best.pth")
    prob_model.to(device)
    prob_model.eval()

    # Load the best results csv file.
    load_path = root / "Results" / "labels_all_best.csv"
    best_df = pd.read_csv(load_path, index_col=0)
    selected_df = best_df[best_df['selected_extra_feature'] == 'predicted_probability']

    # Perform the experiment over models and data examples.
    records = []
    for _, row in tqdm(selected_df.iterrows(), total=len(selected_df)):
        # Load the model.
        model_kwargs = {}
        if row["architecture"] == "GIN":
            model_kwargs = {"train_eps": True}
        elif row["architecture"] == "GAT":
            model_kwargs = {"v2": True}

        label_model = generate_model(
            row["architecture"],
            num_features,
            row["hidden_channels"],
            row["gnn_layers"],
            act=row["activation"],
            jk=row["jk"] if row["jk"] != "none" else None,
            **model_kwargs
        )
        saved_model_dict = torch.load(root / "Models" / f"{row['id']}_best_model.pth", weights_only=False)
        label_model.load_state_dict(saved_model_dict["model_state_dict"])
        label_model.to(device)
        label_model.eval()

        # Calculate the execution time on each data example
        for i in trange(len(prob_data), leave=False, desc=row["architecture"]):
            start = time.perf_counter()
            # First, predict probabilities.
            example = prob_data[i].to(device)
            out = prob_model(example.x, example.edge_index)

            # Second, predict labels.
            example = label_data[i].to(device)
            out = label_model(example.x, example.edge_index)
            end = time.perf_counter()

            # Third, check the MIDS.
            pred = torch.where(out > 0, 1.0, 0.0).numpy()
            A = tg_utils.to_dense_adj(example.edge_index).squeeze().cpu().numpy()
            try:
                correct = int(check_MIDS(A, pred, sum(example.y[:, 0].numpy())))
            except ValueError:
                continue

            records.append({
                "model": row["architecture"],
                "num_nodes": example.num_nodes,
                "num_edges": example.num_edges,
                "correct": correct,
                "predicted": pred.tolist(),
                "ground_truth": example.y.numpy().tolist(),
                "execution_time": (end - start) * 1000
            })

    return pd.DataFrame(records)

In [ ]:
from Utilities.baselines import optimize_Gurobi

def run_Gurobi(label_data):
    records = []

    for i in trange(len(label_data)):
        nx_graph = tg_utils.to_networkx(label_data[i], to_undirected=True)
        sol, execution_time, details = optimize_Gurobi(nx_graph, "MIDS", goal="D", outputFlag=0, single_cpu=False)
        correct = 0 if len(sol) == 0 else 1
        records.append({
            "model": "Gurobi",
            "num_nodes": nx_graph.number_of_nodes(),
            "num_edges": nx_graph.number_of_edges(),
            "correct": correct,
            "execution_time": execution_time
        })

    return pd.DataFrame(records)

In [ ]:
from Utilities.mids_utils import find_MIDS

def run_BK(label_data):
    records = []

    for i in trange(len(label_data)):
        nx_graph = tg_utils.to_networkx(label_data[i], to_undirected=True)
        start = time.perf_counter()
        sol = find_MIDS(nx_graph)
        end = time.perf_counter()
        correct = 0 if len(sol) == 0 else 1
        records.append({
            "model": "Bron-Kerbosch",
            "num_nodes": nx_graph.number_of_nodes(),
            "num_edges": nx_graph.number_of_edges(),
            "correct": correct,
            "execution_time": (end - start) * 1000
        })

    return pd.DataFrame(records)

In [ ]:
from Utilities.baselines import ILPS
from Utilities.mids_utils import check_MIDS
import networkx as nx

def run_ILPS(label_data):
    records = []

    for i in trange(1000):
        nx_graph = tg_utils.to_networkx(label_data[i], to_undirected=True)
        start = time.perf_counter()
        sol = ILPS(nx_graph)
        end = time.perf_counter()

        sol = [1 if x in sol else 0 for x in range(nx_graph.number_of_nodes())]
        A = nx.to_numpy_array(nx_graph)
        correct = int(check_MIDS(A, sol, int(label_data[i].y[:,0].numpy().sum())))
        records.append({
            "model": "ILPS",
            "num_nodes": nx_graph.number_of_nodes(),
            "num_edges": nx_graph.number_of_edges(),
            "correct": correct,
            "execution_time": (end - start) * 1000
        })

    print(f"Correct: {correct}/{len(label_data)}")

    return pd.DataFrame(records)

### Definitions for processing functions

In [2]:
from plotly.validators.scatter.marker import SymbolValidator
import plotly.io as pio

def make_runtime_heatmap(by_nodesize_results, datapath):
    # Pivot the data to create a matrix for the heatmap
    heatmap_data = by_nodesize_results.pivot_table(
        index="num_nodes", columns=["Method"], values="avg"
    )

    # Add hover data for "avg" and "std"
    hover_data = []
    for model in by_nodesize_results["Method"].unique():
        for size in by_nodesize_results["num_nodes"].unique():
            avg_time = by_nodesize_results[
                (by_nodesize_results["Method"] == model) & (by_nodesize_results["num_nodes"] == size)
            ]["avg"].values[0]
            std_dev = by_nodesize_results[
                (by_nodesize_results["Method"] == model) & (by_nodesize_results["num_nodes"] == size)
            ]["std"].values[0]
            hover_data.append((size, model, f"Size: {size}<br>{avg_time:.3f} ± {std_dev:.3f}"))

    hover_df = pd.DataFrame(hover_data, columns=["num_nodes", "Method", "hover"])
    hover_matrix = hover_df.pivot(index="num_nodes", columns="Method", values="hover")
    hover_matrix = hover_matrix.loc[:, list(heatmap_data.columns)]

    # Plot the heatmap
    fig = px.imshow(
        np.log10(heatmap_data),
        title="Average Execution Time by Node Size and Model",
        labels={"color": "Execution Time (ms)"},
        color_continuous_scale=px.colors.sequential.Viridis[::-1]
    )

    # Add custom hover template
    fig.update_traces(text=hover_matrix.values, hovertemplate="%{text}")
    fig.update_layout(
        xaxis=dict(tickmode='array', tickvals=heatmap_data.columns, ticktext=heatmap_data.columns),
        yaxis=dict(tickmode='array', tickvals=heatmap_data.index, ticktext=heatmap_data.index)
    )
    min_z = np.log10(heatmap_data.min(axis=None))
    max_z = np.log10(heatmap_data.max(axis=None))
    fig.update_layout(coloraxis_colorbar=dict(
        tickvals=np.linspace(min_z, max_z, num=6),
        ticktext=[f"{10**x:.3f}" for x in np.linspace(min_z, max_z, num=6)]
    ))

    fig.show()
    fig.write_html(datapath / "runtimes.html")


def make_accuracy_heatmap(by_nodesize_results, datapath):
    acc_heatmap = by_nodesize_results.pivot_table(
        index="num_nodes", columns=["Method"], values="acc"
    )

    # Plot the heatmap
    fig = px.imshow(
        acc_heatmap,
        title="Average Accuracy by Node Size and Model",
        labels={"color": "Accuracy (%)"},
        color_continuous_scale=px.colors.sequential.Viridis[::-1]
    )

    fig.show()
    fig.write_html(datapath / "accuracies.html")


def make_runtime_lineplot(by_nodesize_results, datapath):
    # Create a lineplot with error bars
    fig = px.line(
        by_nodesize_results,
        x="num_nodes",
        y="avg",
        color="Method",
        symbol="Method",
        symbol_sequence=SymbolValidator().values[2::12],
        # error_y=by_nodesize_results["max"] - by_nodesize_results["avg"],
        # error_y_minus=by_nodesize_results["avg"] - by_nodesize_results["min"],
        # title="Execution Time by Node Size and Model",
        labels={"avg": "Average Execution Time (ms)", "num_nodes": "Number of Nodes"},
        log_y=True
    )
    fig.update_traces(marker=dict(size=8))

    # # Create a grouped bar plot with standard deviation
    # fig = px.bar(
    #     by_nodesize_results,
    #     x="num_nodes",
    #     y="avg",
    #     color="Method",
    #     pattern_shape="Method",
    #     error_y=by_nodesize_results["max"] - by_nodesize_results["avg"],
    #     error_y_minus=by_nodesize_results["avg"] - by_nodesize_results["min"],
    #     barmode="group",
    #     title="Execution Time by Node Size and Model",
    #     labels={"avg": "Average Execution Time (ms)", "num_nodes": "Number of Nodes"},
    #     log_y=True
    # )
    # fig.update_traces(error_y=dict(visible=True, symmetric=False, width=0, thickness=1))
    # fig.update_layout(xaxis_type='category')

    fig.update_layout(
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        ),
        font=dict(size=16),
    )

    fig.show()
    fig.write_html(datapath / "runtimes_line.html")
    # fig.write_image("runtimes_line.pdf")
    pio.write_image(fig, datapath / "runtimes_line.pdf", format="pdf", width=750, height=500)



In [3]:
def make_latex_tables(overall_results, by_nodesize_results):
    model_order = ["Gurobi", "Bron-Kerbosch", "MLP", "GCN", "GIN", "GraphSAGE", "GATv2", "GATLinNet"]
    latex_options = dict(position="htb", escape=False)

    overall_results = overall_results.reset_index()

    ## 1. Overall execution time results
    # Combine avg and std columns into a single column with ± sign
    overall_results["Execution Time (ms)"] = overall_results["avg"].round(3).astype(str) + " ± " + overall_results["std"].round(3).astype(str)
    overall_results = overall_results.drop(columns=["avg", "std"])
    overall_results = overall_results.rename(columns={"acc": "Accuracy (%)"})

    # Sort rows with a specified order
    overall_results["Method"] = pd.Categorical(overall_results["Method"], categories=model_order, ordered=True)
    overall_results = overall_results.sort_values("Method")

    table1 = overall_results.drop(columns=["Accuracy (%)"])
    caption = r"Combined average and standard deviation of execution times on the test set containing graphs with \textbf{insert here} nodes."
    overall_times_latex = table1.to_latex(label="tab:total_runtime", index=False, header=True, column_format="l|l", caption=caption, **latex_options)

    ## 2. Execution time results by node size
    # Combine avg and std columns into a single column with ± sign, and rename columns
    by_nodesize_results["Execution Time (ms)"] = by_nodesize_results["avg"].round(3).astype(str) + " ± " + by_nodesize_results["std"].round(3).astype(str)
    by_nodesize_results = by_nodesize_results.drop(columns=["avg", "std"])
    by_nodesize_results = by_nodesize_results.rename(columns={"acc": "Accuracy (%)", "num_nodes": "Number of nodes"})

    # Pivot the dataframe to create the desired table
    nodesize_pivot = by_nodesize_results.pivot_table(
        index='Number of nodes',
        columns=['Method'],
        values='Execution Time (ms)',
        aggfunc="first"
    )
    nodesize_pivot = nodesize_pivot[model_order]

    # Add a row with values of overall execution times from the first table
    overall_times = overall_results.set_index("Method")["Execution Time (ms)"]
    nodesize_pivot.loc["Overall"] = overall_times

    caption = r"Average and standard deviation of execution times on the test set containing graphs with \textbf{insert here} nodes per method and graph size."
    nodesize_times_latex = nodesize_pivot.to_latex(label="tab:size_runtime", column_format="c" * (len(model_order) + 1) , caption=caption, **latex_options)

    ## 3. Accuracy results by node size
    # Pivot the dataframe to create the desired table
    nodesize_pivot = by_nodesize_results.pivot_table(
        index='Number of nodes',
        columns=['Method'],
        values='Accuracy (%)',
        aggfunc="first"
    )
    nodesize_pivot = nodesize_pivot[model_order]

    # Add a row with values of overall accuracies from the first table
    overall_accuracies = overall_results.set_index("Method")["Accuracy (%)"]
    nodesize_pivot.loc["Overall"] = overall_accuracies

    caption = r"Average accuracy on the test set containing graphs with \textbf{insert here} nodes per method and graph size."
    nodesize_accuracy_latex = nodesize_pivot.to_latex(label="tab:size_accuracy", column_format="c" * (len(model_order) + 1), float_format="%.2f", caption=caption, **latex_options)

    return overall_times_latex, nodesize_times_latex, nodesize_accuracy_latex

In [7]:
def process_results(results):
    # Average and standard deviation of execution time for each model over all data examples.
    overall = results.groupby("model")
    overall_avg = overall["execution_time"].mean()
    overall_std = overall["execution_time"].std()
    overall_min = overall["execution_time"].min() # quantile(0.25)
    overall_max = overall["execution_time"].max() # quantile(0.75)
    overall_acc = overall["correct"].mean() * 100

    # Average and standard deviation of execution time for each model over all data examples with the same number of nodes.
    by_nodesize = results.groupby(["model", "num_nodes"])
    by_nodesize_avg = by_nodesize["execution_time"].mean()
    by_nodesize_std = by_nodesize["execution_time"].std()
    by_nodesize_min = by_nodesize["execution_time"].min() # quantile(0.25)
    by_nodesize_max = by_nodesize["execution_time"].max() # quantile(0.75)
    by_nodesize_acc = by_nodesize["correct"].mean() * 100

    return (pd.DataFrame({"avg": overall_avg, "std": overall_std, "min": overall_min, "max": overall_max, "acc": overall_acc}),
            pd.DataFrame({"avg": by_nodesize_avg, "std": by_nodesize_std, "min": by_nodesize_min, "max": by_nodesize_max, "acc": by_nodesize_acc}))


def final_analysis(overall_results, by_nodesize_results, datapath):
    model_order = ["Gurobi", "Bron-Kerbosch", "MLP", "GCN", "GIN", "GraphSAGE", "GAT", "GATLinNet"]

    # Join overall results
    overall_results = pd.concat(overall_results, axis=0, keys=overall_results.keys())
    overall_results = overall_results.droplevel(0)
    overall_results["acc"] = overall_results["acc"].round(2)

    # Join by_nodesize results
    by_nodesize_results = pd.concat(by_nodesize_results, axis=0, keys=by_nodesize_results.keys())

    # Reset index for easier manipulation
    by_nodesize_results = by_nodesize_results.reset_index()
    by_nodesize_results = by_nodesize_results.drop(columns="level_0")
    by_nodesize_results["acc"] = by_nodesize_results["acc"].round(2)

    # Rename the model to Method
    overall_results = overall_results.rename_axis("Method")
    by_nodesize_results = by_nodesize_results.rename(columns={"model": "Method"})

    # Rename GAT row to GATv2

    # Sort index by model order
    overall_results = overall_results.reindex(model_order)
    print(overall_results)

    by_nodesize_results["Method"] = pd.Categorical(by_nodesize_results["Method"], categories=model_order, ordered=True)
    by_nodesize_results["Method"] = by_nodesize_results["Method"].replace({"GAT": "GATv2"})
    by_nodesize_results = by_nodesize_results.sort_values(["Method", "num_nodes"])
    print(by_nodesize_results)

    # Make a heatmap of runtimes
    make_runtime_heatmap(by_nodesize_results, datapath=datapath)

    # Make a heatmap of accuracies
    make_accuracy_heatmap(by_nodesize_results, datapath=datapath)

    # Make a line plot of runtimes
    make_runtime_lineplot(by_nodesize_results, datapath=datapath)

    # Make a line plot of accuracies

    return overall_results, by_nodesize_results

### Save and load

In [ ]:
# Save the results
with open(root / "Results" / "results_small.pkl", "wb") as f:
    pickle.dump((overall_results, by_nodesize_results), f)

In [ ]:
pickle_files = [
    "results_large.pkl",
    "results_small.pkl"
]

overall_results = dict()
by_nodesize_results = dict()

for file in pickle_files:
    with open(root / "Results" / file, "rb") as f:
        overall, by_nodesize = pickle.load(f)
        for key, value in overall.items():
            if key in overall_results:
                overall_results[key] = pd.concat([overall_results[key], value])
            else:
                overall_results[key] = value
        for key, value in by_nodesize.items():
            if key in by_nodesize_results:
                by_nodesize_results[key] = pd.concat([by_nodesize_results[key], value])
            else:
                by_nodesize_results[key] = value

for key, value in overall_results.items():
    overall_results[key] = value.groupby("model").agg({"avg": "mean", "std": "mean", "min": "min", "max": "max", "acc": "mean"})

In [ ]:
with open(root / "Results" / "results_all.pkl", "wb") as f:
    pickle.dump((overall_results, by_nodesize_results), f)

In [5]:
with open(root / "Results" / "results_all.pkl", "rb") as f:
    overall_results, by_nodesize_results = pickle.load(f)

### Run experiments

In [ ]:
# Load the dataset.
prob_data, label_data, num_features = load_dataset(root)

In [ ]:
overall_results = dict()
by_nodesize_results = dict()

# Experiment for GNN models
print("Running GNN models")
GNN_results = run_GNN(root, prob_data, label_data, num_features)
overall_results["GNN"], by_nodesize_results["GNN"] = process_results(GNN_results)

# Experiment for Gurobi
print("Running Gurobi")
Gurobi_results = run_Gurobi(label_data)
overall_results["Gurobi"], by_nodesize_results["Gurobi"] = process_results(Gurobi_results)

# Repeat the experiment for Bron-Kerbosch
print("Running Bron-Kerbosch")
BK_results = run_BK(label_data)
overall_results["BK"], by_nodesize_results["BK"] = process_results(BK_results)

# Repeat the experiment for ILPS
print("Running ILPS")
ILPS_results = run_ILPS(label_data)
overall_results["ILPS"], by_nodesize_results["ILPS"] = process_results(ILPS_results)

### Analyze results

In [8]:
# Analyze the results
overall_concat, by_nodesize_concat = final_analysis(overall_results, by_nodesize_results, datapath=root / "Results")

# Make a latex table
latex_tables = make_latex_tables(overall_concat, by_nodesize_concat)
for table in latex_tables:
    print(table)
    print()

# Save the results
overall_concat.to_csv(root / "Results" / "overall_execution_time.csv")
by_nodesize_concat.to_csv(root / "Results" / "by_nodesize_execution_time.csv")


                     avg        std       min          max     acc
Method                                                            
Gurobi          1.890844   3.553391  0.127041    77.500462  100.00
Bron-Kerbosch  10.317096  56.552528  0.014247  2009.452928  100.00
MLP             1.614324   0.200303  1.397463     7.808938   32.42
GCN             2.431101   0.301011  2.094398    10.726133   23.83
GIN             2.192341   0.333881  1.833192     7.559604   45.56
GraphSAGE       2.023107   0.229427  1.760768     6.951700   52.55
GAT             2.906930   2.768805  2.480285   769.329981   49.54
GATLinNet       2.794112   0.328690  2.410240     9.264585   59.96
        Method  num_nodes       avg       std       min       max     acc
195     Gurobi          4  0.542598       NaN  0.542598  0.542598  100.00
196     Gurobi          5  0.348473  0.098527  0.203029  0.419118  100.00
197     Gurobi          6  0.280064  0.119069  0.127041  0.477296  100.00
198     Gurobi          7  0.31081

/tmp/ipykernel_65929/896800746.py:49: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  by_nodesize_results["Method"] = by_nodesize_results["Method"].replace({"GAT": "GATv2"})
/tmp/ipykernel_65929/2489600565.py:6: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  heatmap_data = by_nodesize_results.pivot_table(


/tmp/ipykernel_65929/2489600565.py:52: FutureWarning:

The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior



\begin{table}[htb]
\caption{Combined average and standard deviation of execution times on the test set containing graphs with \textbf{insert here} nodes.}
\label{tab:total_runtime}
\begin{tabular}{l|l}
\toprule
Method & min & max & Execution Time (ms) \\
\midrule
Gurobi & 0.127041 & 77.500462 & 1.891 ± 3.553 \\
Bron-Kerbosch & 0.014247 & 2009.452928 & 10.317 ± 56.553 \\
MLP & 1.397463 & 7.808938 & 1.614 ± 0.2 \\
GCN & 2.094398 & 10.726133 & 2.431 ± 0.301 \\
GIN & 1.833192 & 7.559604 & 2.192 ± 0.334 \\
GraphSAGE & 1.760768 & 6.951700 & 2.023 ± 0.229 \\
GATLinNet & 2.410240 & 9.264585 & 2.794 ± 0.329 \\
NaN & 2.480285 & 769.329981 & 2.907 ± 2.769 \\
\bottomrule
\end{tabular}
\end{table}


\begin{table}[htb]
\caption{Average and standard deviation of execution times on the test set containing graphs with \textbf{insert here} nodes per method and graph size.}
\label{tab:size_runtime}
\begin{tabular}{ccccccccc}
\toprule
Method & Gurobi & Bron-Kerbosch & MLP & GCN & GIN & GraphSAGE & GATv2 &

/tmp/ipykernel_65929/3046591419.py:28: FutureWarning:

The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior

/tmp/ipykernel_65929/3046591419.py:45: FutureWarning:

The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior

